# Second Assignment of Research Track 2

## Alberto Bono 3962994 

In the terminal execute the command `roslaunch assignment_2_2023 assignment2.launch` to start the simulation.

# Import necessary libraries

In [1]:
import jupyros as jr
import rospy
from std_srvs.srv import Empty
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import time
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from assignment_2_2023.msg import Posvel
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, PlanningActionGoal
from std_srvs.srv import SetBool
from actionlib_msgs.msg import GoalStatus
#
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, HBox, VBox



# Definition of usefull variables

In [2]:
# Initializing of the variables in according to the status of the robot
status_canc = 2
status_reach = 3


# Enable the 'widget' backend for Matplotlib using the IPython interface
get_ipython().run_line_magic('matplotlib', 'widget')

# Enabling interactive plots 

In [3]:
class Goals_button:
    def __init__(self):
        #setting of the button "set_goal_button" to send the goal
        self.set_goal_button = widgets.Button(description="START!", disabled=False, button_style="success", icon='play')
        #setting of the button "cancel_goal_button" to cancel the goal
        self.cancel_goal_button = Button(description="STOP!", disabled=True, button_style="danger", icon='times')
        #setting of the button "kill" to reset the envronment
        self.kill = widgets.Button(value=False, description='Reset enviroment', disabled=False, button_style = 'warning')
        #setting of the spaces "input_x and input_y" for give the position of the target to be sent
        self.input_x = widgets.BoundedFloatText(description='New goal x:', value=False, min=-9, max=9,bar_style = 'info')
        self.input_y = widgets.BoundedFloatText(description='New goal y:', value=False, min=-9, max=9, bar_style = 'info')
        #setting of the function must to activate when the button "set_goal_button" is clicked
        self.set_goal_button.on_click(self.set_new_goal)
        #setting of the function must to activate when the button "cancel_goal_button" is clicked
        self.cancel_goal_button.on_click(self.cancel_current_goal)
        #setting of the fuction must to activate when the button "kill" is clicked
        self.kill.on_click(self.reset_env)
        
       
        
    def set_new_goal(self,button):
       # Create an action client and wait for the server
        client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
        client.wait_for_server()
        # Create a new goal with the input values
        goal = assignment_2_2023.msg.PlanningGoal()
        goal.target_pose.pose.position.x = self.input_x.value
        goal.target_pose.pose.position.y = self.input_y.value
        # Send the goal to the server
        client.send_goal(goal)
        print("Goal was sent!")
        # Disable input fields and buttons appropriately
        self.input_x.disabled = True
        self.input_y.disabled = True
        self.set_goal_button.disabled = True
        self.cancel_goal_button.disabled = False
    def cancel_current_goal(self,button):
        # Cancel the current goal
        client.cancel_goal()
        # Re-enable input fields and buttons appropriately
        self.input_x.disabled = False
        self.input_y.disabled = False
        self.set_goal_button.disabled = False
        self.cancel_goal_button.disabled = True
        print ("Goal was cancel")
    
    def reset_env(self,b):
        # Wait for the reset service and call it
        rospy.wait_for_service('/gazebo/reset_world')
        try:
            reset_world = rospy.ServiceProxy('/gazebo/reset_world', Empty)
            reset_world()
        except rospy.ServiceException:
            print("Failed to reset the environment")




In [4]:


##classe visualizer per plottare il percorso del robot    
class Visualiser:
    def __init__(self,buttons):
        # Initialize lists to store target and robot positions
        self.x_data_target, self.y_data_target = [] , []
        self.x_data_robot, self.y_data_robot = [] , []
        self.pos_label = widgets.Label(value="Current position: ")
        self.vel_label = widgets.Label(value="Current velocity: ")
        self.goal_label = widgets.Label(value="Current goal: ")
        self.goal_received = widgets.Label(value="Goal received: ")
        # Setup matplotlib figures and axes
        output = widgets.Output()
        plt.ioff()
        self.fig, (self.ax1, self.ax2 )= plt.subplots(1,2,figsize=(7,3),constrained_layout=True)
        self.fig.canvas.toolbar_position = 'bottom'
        plt.ion()
        
        with output:
            display(self.fig.canvas)
        
        # Initialize status counts for goals
        self.status_target = [0, 0]
        self.target_labels = ['Canceled', 'Reached']
        self.axis_status=self.ax2.bar(self.target_labels,self.status_target,color=['red','green'])
        # Set titles for the subplots
        self.ax1.set_title('Robot Position')
        self.ax2.set_title('Goals Reached and Canceled')
        # Initialize line objects for plotting robot and target positions
        plt.ioff()
        self.ln_robot, = self.ax1.plot([], [], 'go',label='posizione attuale')
        self.ln_target, = self.ax1.plot([], [], 'yx',label='posizione del goal')
        plt.ion()
        
        self.buttons=buttons
        
        # Setup the layout using ipywidgets
        # First row with input fields and buttons
        up_row =HBox([VBox([self.buttons.input_x, self.buttons.input_y]), VBox([self.buttons.set_goal_button,  self.buttons.cancel_goal_button])]) 
        # Core plot area
        core_up= widgets.Box([output])
        # Last row with reset button
        core_bottom= self.buttons.kill
        # Bottom row with status labels
        bottom_row=VBox([self.pos_label,self.vel_label,self.goal_label])        
        # Combine all parts into the final interface
        interface = widgets.VBox([up_row,core_up,core_bottom,bottom_row])
        # Display the interface
        display(interface)
       
    def plot_init(self):
        self.ax1.set_xlim(10, -10)
        self.ax1.set_ylim(10, -10)
        self.ax1.grid(True)
        self.ax2.set_ylim(0, 10)
        self.ax2.grid(True)
        return self.ln_robot, self.ln_target, self.axis_status
        
    def odom_clbk(self, msg):
        # Initialize the plot settings
        self.y_data_robot.append(msg.pose.pose.position.y)
        self.x_data_robot.append(msg.pose.pose.position.x) 
        self.pos_label.value= f"Current position: {msg.pose.pose.position}"
        self.vel_label.value= f"Current velocity: {msg.twist.twist.linear}"
    def target_clbk(self, msg):
        # Callback for updating robot's position
        self.x_data_target.append(msg.goal.target_pose.pose.position.x)
        self.y_data_target.append(msg.goal.target_pose.pose.position.y)
        self.goal_label.value= f"Current goal: {msg.goal.target_pose.pose.position}"
    def result_clbk(self, msg):
         # Callback for handling goal result
        global status_canc, status_reach
         # Get the status of the goal
        status = msg.status.status
        # Goal canceled
        if status == status_canc:
            self.buttons.input_x.disabled = False
            self.buttons.input_y.disabled = False
            self.buttons.set_goal_button.disabled = False
            self.buttons.cancel_goal_button.disabled = True
            self.status_target[0] += 1
            self.y_data_robot=[]
            self.x_data_robot=[]
            print("Goal reached!")
        # Goal reached
        elif status == status_reach:
            self.buttons.input_x.disabled = False
            self.buttons.input_y.disabled = False
            self.buttons.set_goal_button.disabled = False
            self.buttons.cancel_goal_button.disabled = True
            self.status_target[1] += 1
            self.y_data_robot=[]
            self.x_data_robot=[]
            print ("Goal was cancel ciao sono qui")
    def update_plot(self, frame):
        # Update plot with new data
        self.ln_robot.set_data(self.x_data_robot, self.y_data_robot)
        self.ln_target.set_data(self.x_data_target, self.y_data_target)
        self.axis_status[0].set_height(self.status_target[0])
        self.axis_status[1].set_height(self.status_target[1])
        plt.draw()  # Redraw the plot
        return self.ln_robot, self.ln_target, self.axis_status



In [5]:
# Create an output widget
out = widgets.Output()

# Display the Goals_button and Visualiser instances within the output widget
with out:
    # Create an instance of Goals_button
    buttons = Goals_button()
    # Create an instance of Visualiser, passing the Goals_button instance
    vis = Visualiser(buttons)

# Subscribe the Visualiser instance to ROS topics for receiving data
# '/odom' topic for robot's position and velocity
jr.subscribe('/odom', Odometry, vis.odom_clbk)
# '/reaching_goal/result' topic for handling goal result
jr.subscribe('/reaching_goal/result', assignment_2_2023.msg.PlanningActionResult, vis.result_clbk)
# '/reaching_goal/goal' topic for handling goal target
jr.subscribe('/reaching_goal/goal', assignment_2_2023.msg.PlanningActionGoal, vis.target_clbk)

# Create a FuncAnimation object to continuously update the plot in the Visualiser instance
# 'vis.fig' specifies the figure to be updated
# 'vis.update_plot' specifies the method to update the plot
# 'init_func=vis.plot_init' specifies the initialization function for the plot
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

In [6]:
def main():
    global publisher
    rospy.init_node('goal_sender_node')  
    display(out)

In [9]:
main()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "VBox(children=(HBox(children=(VBox(chi…